In [1]:
# importing libraries
%pip install mediapipe opencv-python matplotlib
import cv2
import numpy as np
import mediapipe as mp
import numpy as np
import time
import matplotlib.pyplot as plt
%matplotlib inline
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

Note: you may need to restart the kernel to use updated packages.


### Capturing the expected poses in a numpy array

In [2]:
# setting workout timer to 30 seconds
timeout = 30
# initializing the Landmarks array
correct_landmarks = np.array([])

In [3]:
workout_time = time.time() + timeout   # 30 seconds from now
cap = cv2.VideoCapture(0)
# setting up mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened() and time.time() < workout_time:
        ret, frame = cap.read()
        # detect pose landmarks and render them on the image
        # convert the image from BGR to RGB (opneCV uses BGR by default)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        # make detection from pose instance
        results = pose.process(image)
        correct_landmarks = np.append(correct_landmarks, results.pose_landmarks)
        # print(results.pose_landmarks)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # print(results)
        # render pose landmarks on the image
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(51,255,51), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(51,153,255), thickness=2, circle_radius=2))
        
        # cv2.imshow('MediaPipe Pose', frame)
        cv2.imshow('MediaPipe Pose', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows() 

In [4]:
correct_landmarks

array([landmark {
         x: 0.436246395111084
         y: 0.20964646339416504
         z: -0.6358551979064941
         visibility: 0.9999966621398926
       }
       landmark {
         x: 0.4520789086818695
         y: 0.17218494415283203
         z: -0.6022113561630249
         visibility: 0.9999785423278809
       }
       landmark {
         x: 0.46073251962661743
         y: 0.17161351442337036
         z: -0.6018756628036499
         visibility: 0.9999887943267822
       }
       landmark {
         x: 0.46960553526878357
         y: 0.17165988683700562
         z: -0.6016863584518433
         visibility: 0.9999730587005615
       }
       landmark {
         x: 0.4175230860710144
         y: 0.17507046461105347
         z: -0.5939683318138123
         visibility: 0.9999728202819824
       }
       landmark {
         x: 0.40673762559890747
         y: 0.1762319952249527
         z: -0.5943613648414612
         visibility: 0.9999872446060181
       }
       landmark {
         

In [6]:
workout_time = time.time() + timeout   # 30 seconds from now
count = 0
cap = cv2.VideoCapture(0)
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        # make detection from pose instance
        current_results = pose.process(image)
        # print(results.pose_landmarks)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        # getting the correct pose landmarks
        correct_pose_landmark = correct_landmarks[count % correct_landmarks.shape[0]]
        count += 1
        #compare the results
        match_count = 0
        for (act, expected) in zip(current_results.pose_landmarks.landmark, correct_pose_landmark.landmark):
            # calculate the distance between the actual and expected landmarks
            dist = np.sqrt((act.x - expected.x)**2 + (act.y - expected.y)**2 + (act.z - expected.z)**2)
            if dist < 0.1:
                match_count += 1
        # calculate the percentage of matched landmarks
        match_percent = (match_count / len(correct_pose_landmark.landmark)) * 100
        text = f"{round(match_percent, 2)}% match - "  
        # render pose landmarks on the image
        if match_percent > 75:
            text += "Correct Pose"
            color = (0, 255, 0)
        else:
            text+= "Incorrect Pose"
            color = (0, 0, 255)
            mp_drawing.draw_landmarks(image, correct_pose_landmark, mp_pose.POSE_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=(51,255,51), thickness=2, circle_radius=2),
                    mp_drawing.DrawingSpec(color=(51,153,255), thickness=2, circle_radius=2))    
            
        mp_drawing.draw_landmarks(image, current_results.pose_landmarks, mp_pose.POSE_CONNECTIONS, 
                                  mp_drawing.DrawingSpec(color=color, thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=color, thickness=2, circle_radius=2))
        cv2.putText(image,text, (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.imshow('MediaPipe Pose', image)
        # render the results from previous cell
        sentence = "No pose detected"
        if cv2.waitKey(10) & 0xFF == ord('q') or time.time() > workout_time:
            break
    cap.release()
    cv2.destroyAllWindows() 